In [1]:
from pgsql_search.loader import HuggingFaceDatasets

ds = HuggingFaceDatasets("UCSC-VLAA/Recap-COCO-30K")
# ds.dataset = ds.dataset.shuffle()
# ds = ds.select(list(range(100)))
ds.save_images("../data/images")
ds = ds.select_columns(["image_filepath", "caption"])
ds.dataset


2024-12-18 23:29:09.423 | INFO     | pgsql_search.loader:__init__:15 - Loading dataset: UCSC-VLAA/Recap-COCO-30K
2024-12-18 23:29:09.424 | INFO     | pgsql_search.loader:save_images:44 - Saving images to folder: ../data/images


Saving images (num_proc=8):   0%|          | 0/30504 [00:00<?, ? examples/s]

Dataset({
    features: ['image_filepath', 'caption'],
    num_rows: 30504
})

In [2]:
df = ds.dataset.to_pandas()
df

,image_filepath,caption
0,../data/images/53120.jpg,Several horses with riders crossing a body of ...
1,../data/images/325873.jpg,A woman and a child sit on the floor and inter...
2,../data/images/217937.jpg,There is an open face sandwich and a fork on t...
3,../data/images/196865.jpg,a pillow a bed a table and two books
4,../data/images/4229.jpg,Three elephants in a field next to each other.
...,...,...
30499,../data/images/289222.jpg,The light shows safe for animal crossing on th...
30500,../data/images/402615.jpg,A crowd of people watching an indoor tennis ma...
30501,../data/images/522959.jpg,"A street corner has a sign, and across the str..."
30502,../data/images/136833.jpg,motorcycles parked on the side of a road near ...


In [3]:
from pgsql_search.database import PostgreSQLDatabase, ColumnType

PostgreSQLDatabase.create_database("my_database")

2024-12-18 23:29:42.890 | INFO     | pgsql_search.database:create_database:344 - Database 'my_database' already exists


In [4]:
with PostgreSQLDatabase("my_database") as db:
    # First, create the table with just an ID column
    db.initialize_table("image_metadata")
    db.add_column("image_filepath", ColumnType.TEXT, nullable=False)
    db.add_column("caption", ColumnType.TEXT, nullable=True)


    db.insert_dataframe(df)

2024-12-18 23:29:42.906 | INFO     | pgsql_search.database:connect:112 - Connected to database
2024-12-18 23:29:42.911 | INFO     | pgsql_search.database:setup_pgvector_extension:129 - pgvector extension initialized
2024-12-18 23:29:42.919 | INFO     | pgsql_search.database:initialize_table:151 - Initialized table 'image_metadata' with ID column 'id'
2024-12-18 23:29:42.922 | INFO     | pgsql_search.database:add_columns:200 - Added 1 new columns to image_metadata
2024-12-18 23:29:42.923 | INFO     | pgsql_search.database:add_columns:200 - Added 1 new columns to image_metadata
2024-12-18 23:29:42.962 | INFO     | pgsql_search.database:insert_dataframe:252 - Inserted batch 1 (1000/30504 rows)
2024-12-18 23:29:42.990 | INFO     | pgsql_search.database:insert_dataframe:252 - Inserted batch 2 (2000/30504 rows)
2024-12-18 23:29:43.028 | INFO     | pgsql_search.database:insert_dataframe:252 - Inserted batch 3 (3000/30504 rows)
2024-12-18 23:29:43.049 | INFO     | pgsql_search.database:insert_